In [1]:
import os
import sys
from sys import platform
import pandas as pd
BASE_PATH = '\\'.join(os.getcwd().split('\\')[:-1]) + '\\' if platform == 'win32' else '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(BASE_PATH+'\src' if platform == 'win32' else BASE_PATH+'/src')
from config.global_config import get_global_config
from utils import get_ds_statistics, get_model_type

In [2]:
_config = get_global_config()
DATA_PATH = os.path.join(BASE_PATH, _config.get('DATASET_PATH'))
RESULTS_PATH = os.path.join(BASE_PATH, _config.get('RESULT_PATH_SHARED'))
if platform == 'win32':
	DATA_PATH = DATA_PATH.replace('/', '\\')
	RESULTS_PATH = RESULTS_PATH.replace('/', '\\')
EMISSIONS_FILE = 'emissions.csv'
METRICS_FILE = 'metrics.csv'
PARAMS_FILE = 'params.csv'

In [3]:
experiments = []
for d in next(os.walk(RESULTS_PATH))[1]:
	for m in next(os.walk(RESULTS_PATH + d))[1]:
		experiments.append(os.path.join(d, m))
experiments = [e for e in experiments if 'SGL' not in e]
experiments = sorted(experiments)

In [4]:
data = pd.DataFrame({
	# Related to dataset
    'n_users': [],
    'n_items': [],
    'n_inter': [],
    'sparsity': [],

    # Related to KG dataset
	'kg_entities': [],
	'kg_relations': [],
	'kg_triples': [],
	'kg_items': [],

	# Related to hardware
    'cpu_cores': [],
    'ram_size': [],
    'is_gpu': [],

	# Related to model
    'model_type': [],

	# General features
    'country_code': [],
	'year': [],

	# Label
	'emissions': []
})

In [5]:
for i, v in enumerate(experiments):
	e = pd.read_csv(os.path.join(RESULTS_PATH, v, EMISSIONS_FILE))
	dataset = v.split("/")[0]
	model = v.split("/")[1]
	stats = get_ds_statistics(os.path.join(DATA_PATH, dataset))
	for _, row in e.iterrows():
		data.loc[len(data)] = [
			stats['n_users'],
			stats['n_items'],
			stats['n_inter'],
			stats['sparsity'],
			stats['kg_entities'],
			stats['kg_relations'],
			stats['kg_triples'],
			stats['kg_items'],
			row['cpu_count'],
			row['ram_total_size'],
			row['gpu_count'],
			get_model_type(model),
			row['country_iso_code'],
			2023,
			row['emissions'],
		]

In [7]:
data.sample(n=5)

,n_users,n_items,n_inter,sparsity,kg_entities,kg_relations,kg_triples,kg_items,cpu_cores,ram_size,is_gpu,model_type,country_code,year,emissions
12,6040,3706,1000209,0.955316,79347,49,385923,3655,12,64.0,1,general,ITA,2023,1.363596e-05
27,6040,3706,1000209,0.955316,79347,49,385923,3655,12,64.0,1,general,ITA,2023,1.717184e-07
17,6040,3706,1000209,0.955316,79347,49,385923,3655,12,64.0,1,knowledge,ITA,2023,2.121586e-04
24,6040,3706,1000209,0.955316,79347,49,385923,3655,12,64.0,1,general,ITA,2023,1.751230e-05
32,6040,3706,1000209,0.955316,79347,49,385923,3655,12,64.0,1,general,ITA,2023,5.986495e-07
